In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd drive/MyDrive/Colab\ Notebooks/LineBot

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/LineBot


In [2]:
!pip install Flask==3.0.0 pyngrok==7.1.2
!pip install line-bot-sdk==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 3.2 MB/s eta 0:00:00


# Ngrok setting

In [3]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# !./ngrok authtoken 2eSTdbOoQhNEvILclqmbzWyulEw_6VGpCwSq7qsCFXfsVuEKF

In [4]:
!./ngrok authtoken 2eSTdbOoQhNEvILclqmbzWyulEw_6VGpCwSq7qsCFXfsVuEKF

/bin/bash: line 1: ./ngrok: Permission denied


In [5]:
ngrok_key = '2eSTdbOoQhNEvILclqmbzWyulEw_6VGpCwSq7qsCFXfsVuEKF'
port = 5000

In [6]:
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://9ac1-34-124-246-58.ngrok-free.app'

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
import torchvision.models as models
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from io import BytesIO

In [8]:
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
    ImageMessage, VideoMessage, AudioMessage
)

import json

In [9]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


In [10]:
MODEL_PATH = 'resnet50_finetuned_weights_0325_6_800.pth'

transform = T.Compose([T.Resize(size=500), T.CenterCrop(size=448), T.ToTensor()])

model = models.resnet50(pretrained=True)
num_flatten = model.fc.in_features
model.fc = nn.Linear(num_flatten, 4)

model.load_state_dict(torch.load(MODEL_PATH))
model = model.to(device)
model.eval()

def prediction(image):
  image = transform(image)
  with torch.no_grad():
    x = image.unsqueeze(0).to(device)
    scores = model(x)

    # 獲取分類項目
    score_max = torch.argmax(scores).item()

    # 獲取各個項目的機率
    predictions = torch.softmax(scores, dim=1)

    # 獲取真實照片的機率
    real_rate = predictions[0][0].item() * 100
    # 獲取AI照片的機率
    ai_rate = 100 - real_rate

    if real_rate > 50:
      return f'這張照片有 {real_rate:.2f}% 的信心度為真實圖片 📷️ 📷️'
    else:
      return f'這張照片有 {ai_rate:.2f}% 的信心度為AI生成圖片 🤖 🤖'

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 87.3MB/s]


In [11]:
CHANNEL_SECRET = '8a35ac9472948491bbc640bf6252cd6b'
CHANNEL_ACCESS_TOKEN = 'SV37EvAcwLe+xTe/2JkHTWxjvJowyXvJPXymBVChrqm9mw8Q8WJBzLR1cRi2F6GiWIVWVimXoAZCbAbHW2duDzFMUmfSipu51eH8uHTCkj7102/wY8WHSXsgZuu9lkLPfhxC6S3FqlVtPsHyIJMGAQdB04t89/1O/w1cDnyilFU='

In [12]:
app = Flask(__name__)

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

@app.route("/callback", methods=['POST'])
def callback():
    body = request.get_data(as_text=True)
    signature = request.headers['X-Line-Signature']
    try:
        json_data = json.loads(body)
        print(json_data)
        handler.handle(body, signature)
        line_bot_api.reply_message(json_data['events'][0]['replyToken'], TextSendMessage('我的功能是判斷圖片是否為AI生成的，所以要傳圖片給我才會有功能喔!🧐'))
    except:
        print(request.args)
    return 'OK'

In [13]:
@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
  message_id = event.message.id
  message_content = line_bot_api.get_message_content(message_id)
  img = Image.open(BytesIO(message_content.content))

  bot_ans = prediction(img)
  line_bot_api.reply_message(event.reply_token, TextSendMessage(bot_ans))

In [ ]:
if __name__ == "__main__":
  app.run(port = port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Apr/2024 10:34:58] "POST /callback HTTP/1.1" 200 -


{'destination': 'Uaf7483d76e808f4f65c56bf47881a338', 'events': []}
ImmutableMultiDict([])
{'destination': 'Uaf7483d76e808f4f65c56bf47881a338', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '502737669470814519', 'quoteToken': 'Pfb2Q8K2FUw4oF6rBEF7Wsunjb6ondloDqSfBRnF5QlaVcMO5Te-07uVy_kcELN4u1HNmV4sU4yWD083GZYcqgYPM17_FOOd15iJ1sI2vrDbniWF-8i8uwBAYnTGKOsMVE8dGIkJ3TUhYIFEMcuf-g', 'text': 'Hi'}, 'webhookEventId': '01HTW26MSDE2JED92610HXH6VZ', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1712486306143, 'source': {'type': 'user', 'userId': 'Ue9389a56b2120ca52f4607a7e173e560'}, 'replyToken': '1727ea68fa3142659f743adc54009a68', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [07/Apr/2024 10:38:27] "POST /callback HTTP/1.1" 200 -


{'destination': 'Uaf7483d76e808f4f65c56bf47881a338', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '502737719199006882', 'quoteToken': 'o_QekFOmg6NTTeaBKERk7FGc2cCe5hJMQkmmvqiyafDqG-9n9r595VsxAIWQxJzvI77KtMsLLO1JNoTZjwfvBQaEreW57rd9BWisUDuAF7HAbeqrL6Ts2o4L0QbIiSpMuYLA6uswP75FODXjp7Fd1g', 'text': 'Hi'}, 'webhookEventId': '01HTW27HSE3K5J0C6CRX4S67PN', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1712486335795, 'source': {'type': 'user', 'userId': 'Ue9389a56b2120ca52f4607a7e173e560'}, 'replyToken': 'f8891a0c1aed4c42b3b11008867b76ee', 'mode': 'active'}]}


...

ImmutableMultiDict([])
{'destination': 'Uaf7483d76e808f4f65c56bf47881a338', 'events': [{'type': 'message', 'message': {'type': 'image', 'id': '502739326925537560', 'quoteToken': 'oeMkkQ0JTcNbmODlZLQ6LazFFm7fDJ1yIWStMCVkkxzz0lxV5E14RRr9VPn5KAMXYN9rhbuM5__p-5R59IADRgnWmMb5zRk6EgIjuAgToPz_RoW8QOG6ZQyDPi2W1ZTE-0ACwXcDCIFeDII9ZPgfaQ', 'contentProvider': {'type': 'line'}}, 'webhookEventId': '01HTW34SHM471199P205EHRZ8F', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1712487294310, 'source': {'type': 'user', 'userId': 'Ue9389a56b2120ca52f4607a7e173e560'}, 'replyToken': '4b20e53a00be4a0ca78f42feb387abce', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [07/Apr/2024 10:54:56] "POST /callback HTTP/1.1" 200 -


ImmutableMultiDict([])
{'destination': 'Uaf7483d76e808f4f65c56bf47881a338', 'events': [{'type': 'message', 'message': {'type': 'image', 'id': '502739405241843828', 'quoteToken': 'qxL410gPrWJXCrqyHESFduGpVdrXpZ7jccrippvfBgvMWTBO6kyjpIXJ8OGcqvJhzj_sSyfIS1RDLs4eccSPGxT0HyIN3K4pfKB4Y7irsMerP7Kw20yOtKGoeff7BzVmdkiR5Wjn5mEvXr3hYzHfBA', 'contentProvider': {'type': 'line'}}, 'webhookEventId': '01HTW367EZ840XR03W9MC5GF12', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1712487341060, 'source': {'type': 'user', 'userId': 'Ue9389a56b2120ca52f4607a7e173e560'}, 'replyToken': '648a463f8a5f4edcbf4a38552b5d9b9e', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [07/Apr/2024 10:55:43] "POST /callback HTTP/1.1" 200 -


ImmutableMultiDict([])


In [ ]:
a = {
    'destination': 'Uaf7483d76e808f4f65c56bf47881a338',
    'events': [{'type': 'message',
             'message': {'type': 'text',
                         'id': '501906363820802100',
                         'quoteToken': 'ET2QUvKt02timZeXs0x7WDYBqIcZeT9hUm3h04A8DrppWFNw7c5bFQtq60nkp9O-6zpCoxmgcWuHeBhjPxxr0NQkGJBaPXlI82hQpRe2PZCnS8sNsRM7foN6IL3iCovYF6ZCUYUg_VG8TAuyMSbgUA',
                         'text': 'P'
                         },
             'webhookEventId': '01HTD9N8Y7P3WEQQX2HHCSYFDM',
             'deliveryContext': {'isRedelivery': False},
             'timestamp': 1711990809362,
             'source': {'type': 'user',
                        'userId': 'Ue9389a56b2120ca52f4607a7e173e560'
                        },
             'replyToken': '768f2f7ac84145319aa17860ede3a7e2',
             'mode': 'active'
             }]
 }

In [ ]:
a['events'][0]['replyToken']

In [ ]:

image = {'destination': 'Uaf7483d76e808f4f65c56bf47881a338',
         'events': [{'type': 'message',
                     'message': {'type': 'image',
                                 'id': '502047787731648933',
                                 'quoteToken': '_iZgEmInd_GuRS_e7qJmCyIpDe6XrWlnSGCyIk2gQFLDXYoSCEfHeUCT2usibYtguEHpmXu3U0-ASWfQwPVqCTxA5D6pRxxOvLyS6VXelam1hcHoFzdnp1dhLhrpIotnEXOdXyWMkBGX4lmt_-Xs3w',
                                 'contentProvider': {'type': 'line'}},
                     'webhookEventId': '01HTFT1S22XPZRH633Q360TJ35',
                     'deliveryContext': {'isRedelivery': False},
                     'timestamp': 1712075105013,
                     'source': {'type': 'user',
                                'userId': 'Ue9389a56b2120ca52f4607a7e173e560'},
                     'replyToken': '5a25883b767a42f59a7e4f446413c9a5',
                     'mode': 'active'
                     }]
         }